In [6]:
!pip install liac-arff

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11727 sha256=65f123fe04976c4418f4ae9bfa1c4108ae7ccc271dd9ef713c67c3cb1fa10a18
  Stored in directory: c:\users\pc user\appdata\local\pip\cache\wheels\00\23\31\5e562fce1f95aabe57f2a7320d07433ba1cd152bcde2f6a002
Successfully built liac-arff


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [70]:
# Initialize an empty list to store the data from each file
data_list = []

# List of ARFF file names
arff_files = ['2017 Q1.arff',
'2017 Q2.arff',
'2017 Q3.arff',
'2017 Q4.arff',
'2017.arff',
'2018 Q1.arff',
'2018 Q2.arff',
'2018 Q3.arff',
'2018 Q4.arff',
'2018.arff',
'2019 Q1.arff',
'2019 Q2.arff',
'2019 Q3.arff',
'2019 Q4.arff',
'2019.arff',
'2020 Q1.arff',
'2020 Q2.arff',
'2020 Q3.arff',
'2020 Q4.arff',
'2020.arff',
'2021 Q1.arff'] 

# Read data from each ARFF file
for file_name in arff_files:
    with open(f'V4 data/{file_name}', 'r') as file:
        lines = file.readlines()
        data_start = False
        current_data = []

        for line in lines:
            if data_start:
                # Assuming data lines start with '@data' and end with the end of the file
                current_data.append(line.strip().split(','))
            elif '@data' in line:
                data_start = True

        data_list.append(current_data)

# Combine data from all files into a single matrix
combined_matrix = [row for file_data in data_list for row in file_data]


def convert_to_float(value):
    try:
        if isinstance(value, str):
            return float(value) if value.replace('.', '', 1).isdigit() or (value[1:].replace('.', '', 1).isdigit() and value[0] == '-') else None
        elif isinstance(value, (int, float)):
            return float(value)
        else:
            return None
    except ValueError:
        return None


combined_matrix = [[convert_to_float(entry) for entry in row] for row in combined_matrix]
X = np.array(combined_matrix)

In [71]:
X

array([[10.0, None, None, ..., None, None, 1.0],
       [22.0, None, 0.0, ..., None, 1.0, 1.0],
       [27.0, None, 0.01, ..., None, None, 1.0],
       ...,
       [427.0, None, None, ..., None, None, 6.0],
       [432.0, None, 0.0, ..., 0.0, 0.0, 6.0],
       [438.0, None, None, ..., None, None, 6.0]], dtype=object)

### PCA from Scratch

In [67]:
from numpy.linalg import eig

class Curativo_PCA:
    def __init__(self, n_components):
        self.n_components = n_components   
        
    def fit(self, X):
        ### Data Standardization
        # getting the mean
        mean = sum(X) / len(X)
        # getting std
        std = (sum((trav - mean) ** 2 for trav in X) / len(X)) ** 0.5
        self.X_std = (X - mean) / std

        ### Covariance Matrix
        # Perform matrix multiplication (@) on self.X_std and its transpose
        covariance_matrix = (self.X_std.T @ self.X_std)/(self.X_std.shape[0]-1)

        ### Eigenvalues and Eigenvectors
        # Eigendecomposition of covariance matrix
        eigenvalues, eigenvectors = eig(covariance_matrix) 
        # Adjusting the eigenvectors (loadings) that are largest in absolute value to be positive
        max_abs_idx = np.argmax(np.abs(eigenvectors), axis=0)
        signs = np.sign(eigenvectors[max_abs_idx, range(eigenvectors.shape[0])])
        eigenvectors = eigenvectors * signs[np.newaxis, :]
        eigenvectors = eigenvectors.T
        
        ### Rearrange in descending order
        # Create eigenpairs from eigenvalues and eigenvectors tuples
        eigenpairs = [(np.abs(eigenvalues[i]), eigenvectors[i, :]) for i in range(len(eigenvalues))]
        # Sort eigenpairs from the highest to the lowest based on eigenvalues magnitude
        eigenpairs.sort(key=lambda x: x[0], reverse=True)
        # Extract sorted eigenvalues and eigenvectors
        eigenvalues_sorted, eigenvectors_sorted = zip(*eigenpairs)
        # Convert the results to numpy arrays
        eigenvalues_sorted = np.array(eigenvalues_sorted)
        eigenvectors_sorted = np.array(eigenvectors_sorted)
        
        ### Principal Components
        # self.n_components == k
        self.components = eigenvectors_sorted[:self.n_components, :]

        ### Explained Variance
        eigenvalues_total = sum(eigenvalues)
        self.explained_variance_ratio = [(i / eigenvalues_total) for i in eigenvalues_sorted[:self.n_components]]
        self.cum_explained_variance = np.cumsum(self.explained_variance_ratio)

        return self

    def transform(self, X):
        ### Data Projection
        X = X.copy()
        X_projection = self.X_std.dot(self.components.T)

        return X_projection



# my_pca = Curativo_PCA(n_components = 2).fit(X)

# print('Components:\n', my_pca.components)
# print('Explained variance ratio from scratch:\n', my_pca.explained_variance_ratio)
# print('Cumulative explained variance from scratch:\n', my_pca.cum_explained_variance)

# X_proj = my_pca.transform(X)
# print('Transformed data shape from scratch:', X_proj.shape)



### Testing My PCA (Using the Given Dataset)

In [77]:
# Extract numerical features (excluding the second column)
numerical_data = X[:, 0].astype(float)

# Create a synthetic second feature with constant values
second_feature = np.ones_like(numerical_data)

# Stack the features horizontally
combined_features = np.column_stack((numerical_data, second_feature))

# Perform PCA with my own pca
my_pca = Curativo_PCA(n_components = 2).fit(X_std)

print('Components:\n', my_pca.components)
print('Explained variance ratio:\n', my_pca.explained_variance_ratio)

cum_explained_variance = np.cumsum(my_pca.explained_variance_ratio)
print('Cumulative explained variance:\n', cum_explained_variance)

X_pca = my_pca.transform(X_std) # Apply dimensionality reduction to X.
print('Transformed data shape:', X_pca.shape)

C:\Users\pc user\AppData\Local\Temp\ipykernel_9104\2423504760.py:13: RuntimeWarning: invalid value encountered in divide
  self.X_std = (X - mean) / std


LinAlgError: Array must not contain infs or NaNs

### PCA from Scikit-Learn (Using the Given Dataset)

In [76]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Extract numerical features (excluding the second column)
numerical_data = X[:, 0].astype(float)

# Create a synthetic second feature with constant values
second_feature = np.ones_like(numerical_data)

# Stack the features horizontally
combined_features = np.column_stack((numerical_data, second_feature))

X_std = StandardScaler().fit_transform(combined_features)

# Perform PCA with the library
pca = PCA(n_components = 2).fit(X_std)

print('Components:\n', pca.components_)
print('Explained variance ratio:\n', pca.explained_variance_ratio_)

cum_explained_variance = np.cumsum(pca.explained_variance_ratio_)
print('Cumulative explained variance:\n', cum_explained_variance)

X_pca = pca.transform(X_std) # Apply dimensionality reduction to X.
print('Transformed data shape:', X_pca.shape)

Components:
 [[1. 0.]
 [0. 1.]]
Explained variance ratio:
 [1. 0.]
Cumulative explained variance:
 [1. 1.]
Transformed data shape: (9450, 2)


### Testing My PCA (Using Iris Dataset)

In [84]:
from sklearn.datasets import load_iris

iris = load_iris()

X = iris['data']
y = iris['target']

my_pca = Curativo_PCA(n_components = 2).fit(X)

print('Components:\n', my_pca.components)
print('Explained variance ratio from scratch:\n', my_pca.explained_variance_ratio)
print('Cumulative explained variance from scratch:\n', my_pca.cum_explained_variance)

X_proj = my_pca.transform(X)
print('Transformed data shape from scratch:', X_proj.shape)

Components:
 [[ 0.52106591 -0.26934744  0.5804131   0.56485654]
 [ 0.37741762  0.92329566  0.02449161  0.06694199]]
Explained variance ratio from scratch:
 [0.7296244541329985, 0.2285076178670178]
Cumulative explained variance from scratch:
 [0.72962445 0.95813207]
Transformed data shape from scratch: (150, 2)


### PCA from Scikit-Learn (Using Iris Dataset)

In [85]:
from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(X)

from sklearn.decomposition import PCA
pca = PCA(n_components = 2).fit(X_std)

print('Components:\n', pca.components_)
print('Explained variance ratio:\n', pca.explained_variance_ratio_)

cum_explained_variance = np.cumsum(pca.explained_variance_ratio_)
print('Cumulative explained variance:\n', cum_explained_variance)

X_pca = pca.transform(X_std) # Apply dimensionality reduction to X.
print('Transformed data shape:', X_pca.shape)

Components:
 [[ 0.52106591 -0.26934744  0.5804131   0.56485654]
 [ 0.37741762  0.92329566  0.02449161  0.06694199]]
Explained variance ratio:
 [0.72962445 0.22850762]
Cumulative explained variance:
 [0.72962445 0.95813207]
Transformed data shape: (150, 2)


### Conclusion

My PCA does not work using the given dataset because of NaN values due to improper preparation of the dataset.

The discrepancy between the Curativo_ PCA code and scikit-learn's PCA library likely stems from the way each approach handles specific scenarios. In my manual implementation, there is an issue with the data standardization process, as evidenced by the warning "invalid value encountered in divide." This suggests that during the standardization, there might be a division by zero or another undefined operation, resulting in NaN or inf values in the data.

On the other hand, scikit-learn's PCA implementation might have incorporated additional checks or mechanisms to handle edge cases more gracefully. It could be robustly managing situations where features have constant values or negligible variation, preventing numerical instability during the standardization process. The library likely includes safeguards to avoid division by zero and ensure a smooth computation. However, using another dataset (the Iris dataset), my PCA works and provides the same results generated by the Scikit-Learn's PCA for several reasons:

Firstly, both implementations follow a standardized procedure, starting with the standardization of data by centering and scaling. Subsequently, they calculate the covariance matrix from the standardized data and perform eigendecomposition to obtain eigenvalues and eigenvectors. Both implementations sort these eigenvalues and eigenvectors in descending order based on eigenvalue magnitude, and adjust the signs of the eigenvectors for consistency. The selection of a specified number of principal components (controlled by n_components) is a commonality, as is the calculation of explained variance ratios and cumulative explained variance. Finally, both implementations transform the original data using the selected principal components. Overall, these shared steps contribute to the expectation that my custom implementation produces results akin to scikit-learn's PCA. However, it must be noted that while both implementations follow the same algorithmic principles, slight differences may arise due to variations in numerical precision and specific implementation details.

### Singular Value Decomposition from Scratch (Using a Random Dataset)

In [82]:
import numpy as np
from numpy.linalg import norm
from random import normalvariate
from math import sqrt

def generate_random_unit_vector(size):
    unnormalized_vector = [normalvariate(0, 1) for _ in range(size)]
    vector_norm = sqrt(sum(v * v for v in unnormalized_vector))
    return [v / vector_norm for v in unnormalized_vector]

def iterate_power_method(matrix_X, epsilon=1e-10):    
    num_rows, num_columns = matrix_X.shape
    start_vector = generate_random_unit_vector(num_columns)
    previous_eigenvector = None
    current_eigenvector = start_vector
    covariance_matrix = np.dot(matrix_X.T, matrix_X)

    # Power iteration until convergence
    iteration_count = 0        
    while True:
        iteration_count += 1
        previous_eigenvector = current_eigenvector
        current_eigenvector = np.dot(covariance_matrix, previous_eigenvector)
        current_eigenvector = current_eigenvector / norm(current_eigenvector)

        if abs(np.dot(current_eigenvector, previous_eigenvector)) > 1 - epsilon:
            return current_eigenvector

def svd_decomposition(matrix_X, epsilon=1e-10):
    num_rows, num_columns = matrix_X.shape
    basis_change_list = []

    for i in range(num_columns):
        modified_matrix = matrix_X.copy()

        for sigma, u, v in basis_change_list[:i]:
            modified_matrix -= sigma * np.outer(u, v) 

        v_vector = iterate_power_method(modified_matrix, epsilon=epsilon)
        u_sigma_vector = np.dot(matrix_X, v_vector)
        sigma_value = norm(u_sigma_vector)  
        u_vector = u_sigma_vector / sigma_value

        basis_change_list.append((sigma_value, u_vector, v_vector))
     
    sigmas, us, v_transposes = [np.array(x) for x in zip(*basis_change_list)]

    return sigmas, us.T, v_transposes


dataset = np.random.random_sample((8, 3))
results = svd_decomposition(dataset)
print("sigmas", results[0])
print("u: data points in new coordinate system", results[1])
print("v transpose: change of basis matrix", results[2]) 


sigmas [2.57223046 1.12178285 0.38969376]
u: data points in new coordinate system [[ 0.31500548 -0.03347854  0.06858326]
 [ 0.45305765  0.08238457 -0.55510515]
 [ 0.32832475  0.28893178 -0.44266707]
 [ 0.21235219  0.02664493  0.07840391]
 [ 0.39056255 -0.42227855 -0.15691987]
 [ 0.26469694  0.53402042  0.36459487]
 [ 0.43199514 -0.55382733  0.4186888 ]
 [ 0.36523464  0.37105087  0.39012782]]
v transpose: change of basis matrix [[ 0.50399956  0.4444461   0.74057553]
 [ 0.81511492 -0.52829626 -0.23767778]
 [ 0.28560832  0.72344366 -0.62853573]]


### Singular Value Decomposition from Scikit-Learn (Using a Random Dataset)

In [83]:
# Singular-value decomposition
from numpy import array
from scipy.linalg import svd

results = svd(dataset)
print("sigmas", results[1])
print("u: data points in new coordinate system", results[0])
print("v transpose: change of basis matrix", results[2]) 

sigmas [2.57223046 1.12178285 0.38969376]
u: data points in new coordinate system [[-0.3150055   0.03347771  0.06858331 -0.20636181  0.00809513 -0.65078201
  -0.14368477 -0.63885353]
 [-0.45305761 -0.08238565 -0.55510527 -0.09441373 -0.49168415  0.20584436
  -0.42646603  0.06997943]
 [-0.32832461 -0.28893255 -0.44266749  0.02925347  0.14459012 -0.10789956
   0.76054828  0.03046899]
 [-0.21235218 -0.02664549  0.07840387  0.9602729  -0.03933807 -0.08077358
  -0.09324217 -0.09570481]
 [-0.39056276  0.42227756 -0.15691926 -0.02902804  0.74688276  0.16815048
  -0.22033788  0.09496784]
 [-0.26469668 -0.53402117  0.36459409 -0.09209808  0.1494365   0.56750926
  -0.03337409 -0.39728365]
 [-0.43199541  0.55382614  0.41868961 -0.07544635 -0.39105299  0.18644024
   0.36996567  0.03326289]
 [-0.36523446 -0.37105189  0.39012728 -0.10249008  0.05137913 -0.3617221
  -0.1543257   0.63946983]]
v transpose: change of basis matrix [[-0.50399864 -0.4444467  -0.74057579]
 [-0.81511563  0.52829539  0.237677

### Conclusion

My manual singular value decomposition (SVD) implementation yields results similar to scikit-learn's SVD for several reasons. Firstly, both implementations follow the same algorithmic principles inherent to SVD, incorporating iterative methods, power iteration, and matrix factorization. The use of a random initialization technique, specifically power iteration with a random unit vector, is a shared approach in both implementations. Additionally, a convergence criterion based on the dot product of current and previous eigenvectors is employed by both to determine the termination of the power iteration process. Given the nature of numerical computations involved in SVD, both implementations may exhibit slight variations due to differences in numerical precision. Both implementations ensure the normalization of eigenvectors and singular vectors for consistency. Adjusting the signs of computed singular vectors is another common practice shared by both implementations. While small differences may exist due to variations in numerical techniques, edge case handling, and specific implementation details, they do not significantly impact the overall similarity in results.
